In [1]:
#Importing Libraries
import boto3
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [ ]:
#Downloading data from the s3 bucket
s3 = boto3.client('s3')
s3.download_file('blossom-data-engs', 'all-us-stocks-tickers-company-info-logos.zip', 'allus.zip')
s3.download_file('blossom-data-engs', 'data-scientist-job-market-in-the-us.zip', 'datascientists.zip')

In [2]:
#Creating a SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
#Read the dataset from local directory
datascientists = spark.read.csv('C:/Users/USER/Desktop/Blossom_Academy/datascientists/alldata.csv', header = True, inferSchema =True)
companies = spark.read.csv('C:/Users/USER/Desktop/Blossom_Academy/companies/companies.csv', header = True, inferSchema =True)

In [4]:
#Checking the number of columns for the companies dataset
companies.columns

['ticker',
 'company name',
 'short name',
 'industry',
 'description',
 'website',
 'logo',
 'ceo',
 'exchange',
 'market cap',
 'sector',
 'tag 1',
 'tag 2',
 'tag 3']

In [5]:
#Checking the number of columns for the datascientists dataset
datascientists.columns

['position', 'company', 'description', 'reviews', 'location']

In [6]:
#Splitting the column location in the Datascientists dataset
datascientists.select('location', F.split(datascientists['location'], ',')[0].alias('City')).show()

+--------------------+--------------------+
|            location|                City|
+--------------------+--------------------+
|                null|                null|
|                 GA.|                 GA.|
|                null|                null|
|            database|            database|
|                null|                null|
| has served as on...| has served as on...|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
| has an open posi...| has an open posi...|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|                null|
|                null|          

In [7]:
#Joining the two Datasets (Company & Datascientists) and renaming one of the columns as comdescription
joindata = datascientists.join(companies.withColumnRenamed('description', 'comdescription'), datascientists['company'] == companies['company name'] )

In [8]:
#Printing the columns of the joined datasets
joindata.columns

['position',
 'company',
 'description',
 'reviews',
 'location',
 'ticker',
 'company name',
 'short name',
 'industry',
 'comdescription',
 'website',
 'logo',
 'ceo',
 'exchange',
 'market cap',
 'sector',
 'tag 1',
 'tag 2',
 'tag 3']

In [9]:
#Showing the first 2 rows of the Joined dataset
joindata.show(5)

+--------------------+---------------+--------------------+-------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------+----------+------+-----+-----+-----+
|            position|        company|         description|            reviews|            location|              ticker|   company name|          short name|            industry|      comdescription|             website|     logo|      ceo| exchange|market cap|sector|tag 1|tag 2|tag 3|
+--------------------+---------------+--------------------+-------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------+----------+------+-----+-----+-----+
|Experience follow...|         design|                code| and defect reviews|                null| The fund general...|         design

### Tokenizing the data to print N-Grams(2)

In [10]:
#Importing libraries for Tokenization and N-Grams
import nltk
from pyspark.ml.feature import Tokenizer, NGram

In [11]:
#Filtering out null values
companies = companies.filter(companies['description'].isNotNull())


In [12]:
#Creating a variable for the Tokenization and also create a new column
token = Tokenizer(inputCol = 'description', outputCol = 'tokenized')

In [13]:
#Dropping null values and applying tokenization
joindata.drop()
token_df = token.transform(joindata)

In [14]:
#Selecting the description and tokenized columns and displaying 5 rows of data
token_df.select('description', 'tokenized').show(5)

+--------------------+--------------------+
|         description|           tokenized|
+--------------------+--------------------+
|                code|            [, code]|
| and planning of ...|[, and, planning,...|
| and planning of ...|[, and, planning,...|
| and moving expen...|[, and, moving, e...|
| and conduct of s...|[, and, conduct, ...|
+--------------------+--------------------+
only showing top 5 rows



In [16]:
#Creating an NGram column and assigning it to bigram
bigram = NGram(n=2, inputCol = 'tokenized', outputCol = 'ngram')
gram = bigram.transform(token_df)

In [17]:
#Showing the 5 rows of the ngram dataset
gram.show(5)

+--------------------+---------------+--------------------+-------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------+----------+------+-----+-----+-----+--------------------+--------------------+
|            position|        company|         description|            reviews|            location|              ticker|   company name|          short name|            industry|      comdescription|             website|     logo|      ceo| exchange|market cap|sector|tag 1|tag 2|tag 3|           tokenized|               ngram|
+--------------------+---------------+--------------------+-------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------+----------+------+-----+-----+-----+--------------------+--------------------+
|Experienc

#### Performing a value count to determine the Frequency 

In [18]:
#Importing Libraries
from pyspark.sql.functions import explode
import pyspark.sql.functions as F

In [19]:
#Printing the Ngram column
gram.select('ngram').limit(1).take(1)

[Row(ngram=[' code'])]

In [21]:
# method for creating the frequency data function...
def create_freq(df, col):
    n=gram.select(col, F.explode('ngram').alias('ngram')).groupBy([col, 'ngram']).count() #exploding the ngrams
    n = n.withColumnRenamed('count','frequency') # changing the column name from count to frequency
    n = n.orderBy(n['frequency'].desc()) # ordering rows by biggest first.
    return n

In [22]:
#creating the freq for industry ngrams
industry_freq = create_freq(gram, 'industry')

In [23]:
industry_freq.show()

Py4JJavaError: An error occurred while calling o216.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 14.0 failed 1 times, most recent failure: Lost task 2.0 in stage 14.0 (TID 19, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$JoinIterator.hasNext(Iterator.scala:212)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$Gener

In [ ]:
from pyspark.sql.functions import countDistinct

In [ ]:
# Change name of columns with alias
#ngram.groupBy("NGram").agg(F.count('NGram')).show()
#ngram.select(countDistinct("NGram")).show()
newngram.select('count')

In [ ]:
newngram.show()

In [ ]:
#import ipython
import nbconvert
import import_ipynb

In [ ]:
import seaborn as sns
sns.distplot(ngram['NGram'])